In [1]:
from pulp import *   # PuLP - Python Library for Linear Programming
import pandas as pd
import numpy as np

n_supply = 100   #number of supply points
n_demand = 4096  #number of demand points

#Importing csv files 
demand_set = pd.read_csv('Demand_Predictions.csv') 
supply_set = pd.read_csv('exisiting_EV_infrastructure_2018.csv')
dist_mat=pd.read_csv('Distance_Matrix.csv')

dist_mat=dist_mat.to_numpy()

#Smax 2018
col=[]
for i in range(supply_set.shape[0]):
    ans=supply_set.iloc[i:i+1,4].values*200+supply_set.iloc[i:i+1,5].values*400
    col.append(int(ans))
col=np.array(col)
supply_set['Smax18']=col
supply_set

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,Smax18
0,0,50.163110,19.412014,23,5,3,2200
1,1,37.336451,58.119225,27,4,7,3600
2,2,46.709232,57.525650,31,6,14,6800
3,3,30.528626,55.379835,26,5,5,3000
4,4,51.521781,35.116755,32,11,6,4600
...,...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4,2200
96,96,30.318396,33.388335,32,5,10,5000
97,97,36.218839,22.235766,32,4,14,6400
98,98,42.936915,38.122442,28,7,5,3400


In [2]:
#Demand values of 2019
demands19=np.array(demand_set['2019'])

#Initialize the PuLP Model
model = LpProblem("Cost-Optimization-Model", LpMinimize)

#Create variable index ids
index_numbers_x = [str(i) for i in range(n_supply*n_demand)]
index_numbers_y = [str(i) for i in range(n_supply)]
index_numbers_x.sort()
index_numbers_y.sort()

#Declare X variable (DS)
DV_variables = LpVariable.matrix("X", index_numbers_x, cat = "Continuous", lowBound= 0 )
dsij_mat19 = np.array(DV_variables).reshape(n_supply,n_demand)

#Empty parking slots
Smax19=[]
park_slot=supply_set['total_parking_slots']-(supply_set['existing_num_SCS']+supply_set['existing_num_FCS'])
park_slot.to_numpy()

#Declare Y variable (Smax)
for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,6].values)
    ub=lb+(park_slot[i]*400)
    temp=LpVariable('Y_{}'.format(index_numbers_y[i]),cat='Integer',lowBound=lb,upBound=ub)
    Smax19.append(temp)
Smax19=np.array(Smax19)

#Customer Dissatisfaction cost 
cost_eq1 = lpSum(dsij_mat19*dist_mat)

S_var=[]
F_var=[]
index_numbers_c = [str(i) for i in range(n_supply)]
index_numbers_c.sort()

#Declare S variable (SCS)
for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,4].values)
    temp=LpVariable('S_{}'.format(index_numbers_c[i]),cat='Integer', lowBound=lb)
    S_var.append(temp)
S_var=np.array(S_var)

#Declare F variable (FCS)
for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,5].values)
    temp=LpVariable('F_{}'.format(index_numbers_c[i]),cat='Integer',lowBound=lb)
    F_var.append(temp)
F_var=np.array(F_var)

#Infrastructure cost
cost_eq2=lpSum(S_var)+1.5*lpSum(F_var)

#Final cost
cost_eq3=cost_eq1+(600*cost_eq2)

#Adding the objective equation to the model (minimize)
model += cost_eq3

#Supply Constraint
#Column summation must be less than or equal to Smax 
for i in range(n_supply):
    model += lpSum(dsij_mat19[i][j] for j in range(n_demand)) <= Smax19[i]-0.01 , "Supply Constraints " + str(i)

# Demand Constraints
#Row summation must be equal to demand forecast
for j in range(n_demand):
    model += lpSum(dsij_mat19[i][j] for i in range(n_supply)) >= demands19[j] , "Demand Constraints " + str(j)

#Smax constraint
for i in range(n_supply):
    constraint = (S_var[i]*200)+(F_var[i]*400) == Smax19[i]
    model += constraint

model.writeLP('FinalModel.lp')

In [2]:
model.solve(PULP_CBC_CMD())
status =  LpStatus[model.status]
print(status)

Optimal


In [3]:
print("Total Cost:", model.objective.value())

Total Cost: 2586519.7978973263


In [4]:
allVar=[]
for v in model.variables():
    k=v.value()
    allVar.append(k)

p=0
F_var=[]
S_var=[]
for i in range(n_supply):
    F_var.append(int(allVar[p]))
    p+=1
for i in range(n_supply):
    S_var.append(int(allVar[p]))
    p+=1

dsij_mat19=np.zeros((100,4096))
    
for i in range(n_supply):
    for j in range(n_demand):
        dsij_mat19[i][j]=allVar[p]
        p+=1

Smax19=[]
for i in range(n_supply):
    Smax19.append(int(allVar[p]))
    p+=1
Smax19=np.array(Smax19)

dsij_mat19=pd.DataFrame(dsij_mat19)
dsij_mat19.to_csv('Total_Dsij_19_v2.csv') #Export DSij matrix

In [5]:
#Checking the Demand constraint (row summation)
print(sum(dsij_mat19[0]))
#Comparing with demand value
print(demands19[0])

15.08854
15.088539719581604


In [6]:
#Checking the Supply constraint (column summation)
print(sum(dsij_mat19.iloc[0,:].values))
#Comparing with supply value
print(Smax19[0])

7799.989976500003
7800


In [7]:
supply_set['Smax19']=Smax19
supply_set['SCS_19']=S_var
supply_set['FCS_19']=F_var

supply_set.to_csv('Total_infra_19_v2.csv') #Export new infrastructure